In [ ]:
# Execute SQL command and store the results
schema_names = sqlContext.sql("SHOW DATABASES").collect()

# Extract schema names into a list
schemas = [row.databaseName for row in schema_names]

# Initialize an empty list for all grant data
all_grants = []

for schema in schemas:
    # Generate SQL command
    sql_command = f"SHOW GRANTS ON SCHEMA `{schema}`"

    # Execute SQL command and get the results
    grants = sqlContext.sql(sql_command).collect()

    # Store results with schema name
    for grant in grants:
        all_grants.append((schema, grant.Principal, grant.ActionType, grant.ObjectType, grant.ObjectKey))

# Define the schema for the DataFrame
from pyspark.sql.types import StructType, StructField, StringType

schema = StructType([
    StructField("Schema", StringType(), True),
    StructField("Principal", StringType(), True),
    StructField("ActionType", StringType(), True),
    StructField("ObjectType", StringType(), True),
    StructField("ObjectKey", StringType(), True)
])

# Convert the list to a DataFrame
grants_df = spark.createDataFrame(all_grants, schema)

# Write the DataFrame to a Hive table
grants_df.write.mode('overwrite').saveAsTable("hive_metastore_grants")
